In [4]:
# Copyright 2025 EPFL – Apache 2.0
#
# Process *all* Penn-Action videos:
#   RGB   → Cosmos tokens  →  output/dump/train/tok_rgb/0001/00001.npy …
#   Pose  → Cosmos tokens  →  output/dump/train/tok_pose/0001/00001.npy …
#   Coords → .npy                        idem
#   Caption → .json                      idem
# --------------------------------------------------------------------

import os, json
import numpy as np
import torch
from pathlib import Path
import torchvision.transforms.functional as TF
from huggingface_hub import snapshot_download
from PIL import Image, ImageDraw
import scipy.io
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration
from cosmos_tokenizer.image_lib import ImageTokenizer

# ---------------- paths ----------------
frames_root  = "../project/penn_action_raw/Penn_Action/frames"
labels_root  = "../project/penn_action_raw/Penn_Action/labels"
out_root     = "../project/output/"                  # master output folder

# create modality sub-dirs once; each video gets its own sub-folder inside
for sub in ("tok_rgb", "coords", "captions"):
    (Path(out_root)/sub).mkdir(parents=True, exist_ok=True)

# --------------- models ----------------
device = "cuda" if torch.cuda.is_available() else "cpu"
snapshot_download(
    repo_id='nvidia/Cosmos-0.1-Tokenizer-DI16x16', 
    local_dir='/tmp/nvidiaxxxx/Cosmos-0.1-Tokenizer-DI16x16'
)
image_tokenizer = ImageTokenizer(
    checkpoint_enc="/tmp/nvidiaxxxx/Cosmos-0.1-Tokenizer-DI16x16/encoder.jit",
    checkpoint_dec="/tmp/nvidiaxxxx/Cosmos-0.1-Tokenizer-DI16x16/decoder.jit",
).to(device)

processor  = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
).to(device)

# ------------- helpers -----------------
def encode_rgb(img: Image.Image) -> np.ndarray:
    ten = TF.to_tensor(img).unsqueeze(0).to(device) * 2 - 1
    with torch.inference_mode():
        tok, _ = image_tokenizer.encode(ten)
    return tok.squeeze(0).cpu().short().numpy()

def draw_pose(x, y, vis, size=640):
    edges = [(1,2),(1,3),(2,4),(3,5),(4,6),(7,8),(7,9),
             (8,10),(9,11),(10,12)]
    canvas = Image.new("RGBA", (size, size), (0,0,0,255))
    drw    = ImageDraw.Draw(canvas)
    for j in range(13):
        if vis[j]:
            cx, cy = int(x[j]), int(y[j]); r = 5 if j==0 else 3
            drw.ellipse((cx-r, cy-r, cx+r, cy+r),
                        outline="red" if j==0 else None,
                        width=2 if j==0 else 0,
                        fill=(255,0,0,255) if j else None)
    for j1,j2 in edges:
        if vis[j1] and vis[j2]:
            drw.line([(int(x[j1]),int(y[j1])),(int(x[j2]),int(y[j2]))],
                     fill="red", width=2)
    if vis[0] and vis[7] and vis[8]:
        hip = tuple(np.mean([[x[7],y[7]],[x[8],y[8]]],0).astype(int))
        drw.line([(int(x[0]),int(y[0])), hip], fill="red", width=2)
    return canvas.convert("RGB")

def encode_pose(x,y,vis):
    return encode_rgb(draw_pose(x,y,vis))

def caption_image(img):
    inp = processor(images=img, return_tensors="pt").to(device)
    out = blip_model.generate(**inp, max_length=30)
    return processor.decode(out[0], skip_special_tokens=True)

# Set the specific video directory to process
video_to_process = "0003"  # Replace this with the actual video name

# ---------- main loop for a single video --------------
frame_dir = Path(frames_root) / video_to_process

# Ensure the label file exists for the specified video
label_file = Path(labels_root) / f"{video_to_process}.mat"
if not label_file.exists():
    print(f"⚠️  Skip {video_to_process}: no label file")
else:
    # ---- load MATLAB struct once per video ----
    mat = scipy.io.loadmat(label_file, squeeze_me=True, struct_as_record=False)
    x_all, y_all, vis_all = mat['x'], mat['y'], mat['visibility'].astype(bool)
    T = int(mat['nframes'])
    H0, W0, _ = mat['dimensions']
    scale_x, scale_y = 640.0 / W0, 640.0 / H0

    frame_files = sorted(frame_dir.glob("*.jpg"))
    assert len(frame_files) == T, f"{video_to_process}: frame/label mismatch"

    # ensure per-video output dirs exist
    for sub in ("tok_rgb", "coords", "captions"):
        (Path(out_root) / sub / video_to_process).mkdir(parents=True, exist_ok=True)

    # Process each frame in the video
    for i, fpath in enumerate(tqdm(frame_files, desc=video_to_process, leave=False)):
        stem = f"{i+1:05d}"
        rgb_out = Path(out_root) / "tok_rgb" / video_to_process / f"{stem}.npy"
        coords_out = Path(out_root) / "tok_rgb" / video_to_process / f"{stem}.npy"
        captions_out = Path(out_root) / "tok_rgb" / video_to_process / f"{stem}.npy"

        # ---------- SKIP if already done ----------
        # if rgb_out.exists() and captions_out.exists() and coords_out.exists() :
        #   continue

        # ---------- RGB ----------
        img = Image.open(fpath).resize((640, 640))
        np.save(rgb_out, encode_rgb(img))

        # ---------- Pose & coords ----------
        x = x_all[i] * scale_x
        y = y_all[i] * scale_y
        vis = vis_all[i]
        
        coords = np.stack([x / 640.0, y / 640.0, vis.astype(float)])
        np.save(Path(out_root) / "coords" / video_to_process / f"{stem}.npy", coords)

        # ---------- Caption ----------
        cap = caption_image(img)
        with open(Path(out_root) / "captions" / video_to_process / f"{stem}.json", "w") as fp:
            json.dump({"video": video_to_process, "frame": stem, "caption": cap}, fp, indent=2)

print(f"✅ All frames of video {video_to_process} processed → {out_root}")



Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

0003:   1%|█▉                                                                                                                                                                     | 1/85 [00:00<00:23,  3.53it/s]

OK


0003:   2%|███▉                                                                                                                                                                   | 2/85 [00:01<01:04,  1.29it/s]

OK


...

0003:  98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████    | 83/85 [00:20<00:00,  4.51it/s]

OK
OK


0003:  99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████  | 84/85 [00:20<00:00,  4.55it/s]

OK


OK
✅ All frames of video 0003 processed → ../project/output/


In [1]:
import os, json
import numpy as np
import torch
from pathlib import Path
import torchvision.transforms.functional as TF
from huggingface_hub import snapshot_download
from PIL import Image
import scipy.io
from tqdm import tqdm
from cosmos_tokenizer.image_lib import ImageTokenizer

# ---------------- paths ----------------
frames_root  = "../project/penn_action_raw/Penn_Action/frames"
labels_root  = "../project/penn_action_raw/Penn_Action/labels"
out_root     = "../project/output/"                  # master output folder

# create modality sub-dirs once; each video gets its own sub-folder inside
Path(out_root, "tok_rgb").mkdir(parents=True, exist_ok=True)

# --------------- models ----------------
device = "cuda" if torch.cuda.is_available() else "cpu"
snapshot_download(
    repo_id='nvidia/Cosmos-0.1-Tokenizer-DI16x16', 
    local_dir='/tmp/nvidiaxxxx/Cosmos-0.1-Tokenizer-DI16x16'
)
image_tokenizer = ImageTokenizer(
    checkpoint_enc="/tmp/nvidiaxxxx/Cosmos-0.1-Tokenizer-DI16x16/encoder.jit",
    checkpoint_dec="/tmp/nvidiaxxxx/Cosmos-0.1-Tokenizer-DI16x16/decoder.jit",
).to(device)

# ------------- helpers -----------------
def encode_rgb(img: Image.Image) -> np.ndarray:
    """Tokenize the image using Cosmos tokenizer."""
    img_tensor = TF.to_tensor(img).to(device).unsqueeze(0) * 2 - 1  # Normalize to [-1, 1]
    tokens, _ = image_tokenizer.encode(img_tensor)
    tokens = tokens.cpu().numpy()
    return tokens

def process_video(video_to_process):
    # ---------- main loop for a single video --------------
    frame_dir = Path(frames_root) / video_to_process

    # Ensure the label file exists for the specified video
    label_file = Path(labels_root) / f"{video_to_process}.mat"
    if not label_file.exists():
        print(f"⚠️  Skip {video_to_process}: no label file")
        return

    # ---- load MATLAB struct once per video ----
    mat = scipy.io.loadmat(label_file, squeeze_me=True, struct_as_record=False)
    x_all, y_all, vis_all = mat['x'], mat['y'], mat['visibility'].astype(bool)
    T = int(mat['nframes'])
    H0, W0, _ = mat['dimensions']
    scale_x, scale_y = 640.0 / W0, 640.0 / H0

    frame_files = sorted(frame_dir.glob("*.jpg"))
    assert len(frame_files) == T, f"{video_to_process}: frame/label mismatch"

    # ensure per-video output dirs exist
    Path(out_root, "tok_rgb", video_to_process).mkdir(parents=True, exist_ok=True)

    # Process each frame in the video
    for i, fpath in enumerate(tqdm(frame_files, desc=video_to_process, leave=False)):
        stem = f"{i+1:05d}"
        rgb_out = Path(out_root) / "tok_rgb" / video_to_process / f"{stem}.npy"

        # ---------- SKIP if already done ----------
        if rgb_out.exists():
            print(video_to_process, "Already processed")
            continue

        # ---------- RGB ----------
        img = Image.open(fpath).resize((640, 640))
        np.save(rgb_out, encode_rgb(img))  # Save the tokenized RGB frame

    print(f"✅ All frames of video {video_to_process} processed → {out_root}")


# ---------- Process all videos --------------
video_ids = [ "0001", "0094",    "0187",  "0280",  "0363", "0450", "0535",  "0617", "0695",  "0790" , "0882",  "0969",  "1059",  "1147",  "1241",  "1337",  "1420", "1517", "1618",  "1712",  "1792",  "1881" , "1972",  "2060", "2153",  "2252",
"0003",  "0095",  "0188", "0281",  "0366",  "0452",  "0537",  "0618", "0697",  "0791", "0883",  "0970" ,"1061",  "1149",  "1242",  "1338",  "1422",  "1519",  "1621", "1713", "1793",  "1882",  "1974",  "2062" , "2154",  "2253",
"0004",  "0096",  "0190", "0282",  "0367",  "0453",  "0539",  "0619", "0698",  "0793", "0884",  "0973" ,"1062",  "1150",  "1243",  "1339",  "1425",  "1520",  "1622", "1714", "1794",  "1885",  "1975",  "2063" , "2155",  "2254",
"0006",  "0098",  "0191", "0283",  "0368",  "0454",  "0540",  "0620", "0699",  "0797", "0885",  "0974" ,"1063",  "1153",  "1244",  "1340",  "1426",  "1521",  "1625", "1716", "1795",  "1886",  "1977",  "2064" , "2156",  "2256",
"0007",  "0099",  "0192", "0284",  "0369",  "0455",  "0541",  "0622", "0701",  "0798", "0888",  "0975" ,"1065",  "1155",  "1245",  "1341",  "1427",  "1524",  "1626", "1718", "1797",  "1887",  "1978",  "2066" , "2157",  "2257",
"0008",  "0100",  "0193", "0285",  "0371",  "0456",  "0542",  "0623", "0703",  "0799", "0889",  "0976" ,"1066",  "1156",  "1246",  "1342",  "1428",  "1530",  "1627", "1719", "1798",  "1888",  "1980",  "2068" , "2162",  "2258",
"0009",  "0101",  "0195", "0286",  "0372",  "0457",  "0543",  "0624", "0704",  "0803", "0890",  "0977" ,"1068",  "1157",  "1248",  "1343",  "1433",  "1531",  "1628", "1720", "1799",  "1889",  "1981",  "2069" , "2163",  "2259",
"0012",  "0102",  "0199", "0287",  "0373",  "0459",  "0544",  "0626", "0705",  "0804", "0891",  "0978" ,"1069",  "1160",  "1249",  "1344",  "1436",  "1532",  "1629", "1721", "1801",  "1891",  "1983",  "2070" , "2164",  "2262",
"0013",  "0103",  "0202", "0289",  "0377",  "0460",  "0545",  "0627", "0706",  "0806", "0892",  "0979" ,"1071",  "1161",  "1250",  "1349",  "1437",  "1533",  "1630", "1723", "1802",  "1893",  "1984",  "2071" , "2165",  "2263",
"0014",  "0105",  "0204", "0291",  "0378",  "0461",  "0547",  "0628", "0707",  "0807", "0893",  "0980" ,"1075",  "1165",  "1252",  "1350",  "1438",  "1535",  "1631", "1724", "1803",  "1894",  "1986",  "2072" , "2167",  "2264",
"0016",  "0106",  "0205", "0292",  "0379",  "0463",  "0548",  "0629", "0708",  "0808", "0894",  "0981" ,"1077",  "1166",  "1253",  "1352",  "1439",  "1537",  "1632", "1725", "1805",  "1895",  "1987",  "2073" , "2168",  "2265",
"0017",  "0107",  "0206", "0293",  "0380",  "0465",  "0550",  "0630", "0711",  "0809", "0896",  "0982" ,"1078",  "1167",  "1255",  "1354",  "1440",  "1538",  "1635", "1726", "1806",  "1896",  "1989",  "2076" , "2169",  "2266",
"0020",  "0109",  "0208", "0294",  "0382",  "0466",  "0551",  "0631", "0712",  "0811", "0897",  "0987" ,"1079",  "1168",  "1260",  "1355",  "1441",  "1539",  "1636", "1728", "1808",  "1899",  "1990",  "2077" , "2171",  "2267",
"0021",  "0110",  "0209", "0295",  "0383",  "0467",  "0552",  "0632", "0713",  "0813", "0898",  "0988" ,"1083",  "1169",  "1261",  "1356",  "1442",  "1540",  "1637", "1729", "1809",  "1900",  "1991",  "2079" , "2172",  "2268",
"0022",  "0111",  "0211", "0296",  "0384",  "0469",  "0554",  "0633", "0714",  "0814", "0899",  "0989" ,"1084",  "1171",  "1262",  "1357",  "1443",  "1541",  "1638", "1730", "1810",  "1901",  "1993",  "2080" , "2173",  "2269",
"0026",  "0112",  "0213", "0297",  "0385",  "0470",  "0555",  "0634", "0715",  "0815", "0900",  "0990" ,"1085",  "1172",  "1265",  "1358",  "1446",  "1544",  "1639", "1732", "1811",  "1902",  "1994",  "2081" , "2174",  "2270",
"0027",  "0113",  "0215", "0298",  "0386",  "0472",  "0557",  "0635", "0717",  "0816", "0901",  "0991" ,"1086",  "1173",  "1266",  "1359",  "1447",  "1545",  "1641", "1733", "1812",  "1904",  "1997",  "2082" , "2175",  "2273",
"0029",  "0114",  "0217", "0299",  "0387",  "0473",  "0558",  "0636", "0719",  "0817", "0902",  "0992" ,"1087",  "1174",  "1267",  "1360",  "1448",  "1546",  "1642", "1734", "1813",  "1905",  "1999",  "2083" , "2176",  "2274",
"0030",  "0115",  "0218", "0301",  "0390",  "0474",  "0559",  "0637", "0720",  "0818", "0903",  "0993" ,"1088",  "1176",  "1268",  "1361",  "1449",  "1547",  "1643", "1735", "1816",  "1907",  "2000",  "2085" , "2177",  "2275",
"0032",  "0117",  "0219", "0302",  "0391",  "0475",  "0560",  "0638", "0721",  "0819", "0905",  "0994" ,"1089",  "1178",  "1269",  "1362",  "1450",  "1549",  "1644", "1737", "1817",  "1909",  "2001",  "2088" , "2181",  "2276",
"0034",  "0118",  "0220", "0303",  "0392",  "0476",  "0562",  "0639", "0722",  "0821", "0906",  "0995" ,"1091",  "1179",  "1270",  "1363",  "1452",  "1551",  "1646", "1740", "1819",  "1910",  "2002",  "2089" , "2182",  "2278",
"0036",  "0119",  "0221", "0304",  "0393",  "0479",  "0563",  "0640", "0725",  "0823", "0907",  "0996" ,"1092",  "1180",  "1272",  "1365",  "1453",  "1553",  "1647", "1741", "1820",  "1911",  "2003",  "2090" , "2183",  "2281",
"0039",  "0123",  "0222", "0305",  "0395",  "0481",  "0565",  "0643", "0727",  "0824", "0909",  "0998" ,"1093",  "1181",  "1273",  "1366",  "1454",  "1555",  "1648", "1742", "1822",  "1912",  "2006",  "2091" , "2184",  "2282",
"0040",  "0124",  "0223", "0306",  "0396",  "0482",  "0566",  "0645", "0730",  "0825", "0914",  "0999" ,"1094",  "1182",  "1275",  "1368",  "1456",  "1556",  "1650", "1743", "1823",  "1913",  "2008",  "2093" , "2187",  "2283",
"0041",  "0126",  "0226", "0309",  "0397",  "0485",  "0567",  "0647", "0732",  "0826", "0915",  "1000" ,"1095",  "1184",  "1276",  "1369",  "1457",  "1557",  "1651", "1745", "1825",  "1914",  "2009",  "2094" , "2188",  "2284",
"0042",  "0128",  "0227", "0311",  "0398",  "0486",  "0568",  "0648", "0733",  "0828", "0918",  "1001" ,"1096",  "1186",  "1277",  "1370",  "1458",  "1559",  "1652", "1747", "1832",  "1915",  "2011",  "2097" , "2189",  "2285",
"0043",  "0129",  "0230", "0312",  "0399",  "0488",  "0570",  "0649", "0735",  "0829", "0921",  "1002" ,"1097",  "1188",  "1279",  "1372",  "1459",  "1560",  "1653", "1749", "1834",  "1917",  "2012",  "2098" , "2190",  "2287",
"0045",  "0130",  "0232", "0313",  "0402",  "0489",  "0571",  "0650", "0736",  "0831", "0922",  "1003" ,"1098",  "1189",  "1280",  "1374",  "1460",  "1561",  "1654", "1750", "1835",  "1918",  "2013",  "2099" , "2191",  "2288",
"0047",  "0131",  "0233", "0314",  "0404",  "0490",  "0572",  "0652", "0739",  "0832", "0923",  "1004" ,"1099",  "1190",  "1281",  "1375",  "1462",  "1563",  "1655", "1751", "1836",  "1921",  "2015",  "2100" , "2193",  "2289",
"0048",  "0132",  "0234", "0316",  "0405",  "0491",  "0573",  "0654", "0740",  "0833", "0924",  "1006" ,"1102",  "1192",  "1282",  "1376",  "1464",  "1564",  "1657", "1752", "1838",  "1922",  "2016",  "2102" , "2195",  "2290",
"0049",  "0134",  "0235", "0317",  "0406",  "0492",  "0575",  "0655", "0741",  "0835", "0926",  "1008" ,"1103",  "1193",  "1284",  "1377",  "1465",  "1566",  "1659", "1753", "1839",  "1923",  "2017",  "2104" , "2196",  "2292",
"0051",  "0135",  "0236", "0319",  "0407",  "0493",  "0576",  "0656", "0742",  "0836", "0928",  "1009" ,"1104",  "1194",  "1285",  "1378",  "1468",  "1568",  "1660", "1754", "1840",  "1924",  "2018",  "2105" , "2197",  "2293",
"0052",  "0139",  "0237", "0320",  "0408",  "0494",  "0578",  "0658", "0745",  "0837", "0930",  "1010" ,"1105",  "1196",  "1287",  "1379",  "1470",  "1569",  "1661", "1755", "1841",  "1925",  "2019",  "2106" , "2199",  "2297",
"0053",  "0141",  "0238", "0321",  "0409",  "0495",  "0579",  "0659", "0746",  "0838", "0931",  "1011" ,"1106",  "1197",  "1289",  "1382",  "1471",  "1574",  "1662", "1756", "1842",  "1927",  "2020",  "2111" , "2200",  "2298",
"0054",  "0142",  "0239", "0322",  "0410",  "0496",  "0580",  "0660", "0747",  "0840", "0932",  "1012" ,"1107",  "1199",  "1290",  "1383",  "1472",  "1577",  "1665", "1757", "1843",  "1929",  "2021",  "2112" , "2201",  "2299",
"0056",  "0143",  "0240", "0323",  "0411",  "0498",  "0582",  "0661", "0748",  "0842", "0933",  "1013" ,"1109",  "1201",  "1293",  "1384",  "1473",  "1578",  "1666", "1759", "1844",  "1930",  "2022",  "2113" , "2206",  "2300",
"0057",  "0144",  "0241", "0324",  "0412",  "0499",  "0583",  "0662", "0750",  "0843", "0934",  "1015" ,"1111",  "1202",  "1294",  "1385",  "1474",  "1580",  "1667", "1761", "1848",  "1931",  "2023",  "2116" , "2207",  "2302",
"0059",  "0146",  "0242", "0325",  "0414",  "0500",  "0584",  "0663", "0752",  "0844", "0935",  "1016" ,"1112",  "1203",  "1295",  "1386",  "1475",  "1581",  "1668", "1762", "1849",  "1932",  "2024",  "2117" , "2208",  "2303",
"0060",  "0147",  "0243", "0326",  "0417",  "0501",  "0586",  "0665", "0755",  "0845", "0936",  "1018" ,"1113",  "1205",  "1296",  "1388",  "1478",  "1582",  "1670", "1763", "1850",  "1935",  "2026",  "2119" , "2209",  "2305",
"0061",  "0149",  "0244", "0327",  "0418",  "0502",  "0587",  "0666", "0756",  "0847", "0938",  "1019" ,"1115",  "1206",  "1297",  "1389",  "1479",  "1584",  "1671", "1764", "1852",  "1936",  "2028",  "2120" , "2210",  "2306",
"0062",  "0150",  "0245", "0329",  "0419",  "0503",  "0588",  "0667", "0757",  "0849", "0939",  "1020" ,"1116",  "1207",  "1299",  "1390",  "1482",  "1588",  "1673", "1766", "1853",  "1937",  "2031",  "2124" , "2212",  "2308",
"0063",  "0152",  "0248", "0330",  "0420",  "0505",  "0589",  "0669", "0758",  "0850", "0941",  "1021" ,"1117",  "1208",  "1300",  "1391",  "1483",  "1590",  "1674", "1767", "1855",  "1938",  "2033",  "2125" , "2216",  "2309",
"0064",  "0154",  "0249", "0331",  "0421",  "0507",  "0590",  "0670", "0759",  "0851", "0942",  "1022" ,"1118",  "1211",  "1302",  "1392",  "1484",  "1593",  "1677", "1768", "1856",  "1939",  "2034",  "2126" , "2218",  "2311",
"0066",  "0155",  "0250", "0332",  "0424",  "0509",  "0591",  "0671", "0760",  "0852", "0943",  "1024" ,"1119",  "1213",  "1303",  "1394",  "1487",  "1594",  "1680", "1769", "1858",  "1942",  "2035",  "2127" , "2219",  "2312",
"0067",  "0156",  "0254", "0333",  "0425",  "0511",  "0592",  "0672", "0761",  "0853", "0945",  "1025" ,"1121",  "1214",  "1304",  "1397",  "1490",  "1595",  "1681", "1770", "1859",  "1945",  "2036",  "2129" , "2220",  "2314",
"0068",  "0157",  "0255", "0336",  "0426",  "0512",  "0597",  "0673", "0762",  "0854", "0946",  "1028" ,"1123",  "1215",  "1305",  "1398",  "1491",  "1596",  "1682", "1771", "1860",  "1947",  "2038",  "2130" , "2222",  "2315",
"0069",  "0159",  "0256", "0337",  "0430",  "0513",  "0598",  "0674", "0763",  "0855", "0947",  "1029" ,"1125",  "1216",  "1306",  "1399",  "1493",  "1597",  "1683", "1772", "1862",  "1948",  "2039",  "2131" , "2223",  "2317",
"0071",  "0160",  "0257", "0338",  "0431",  "0514",  "0599",  "0675", "0764",  "0859", "0951",  "1030" ,"1127",  "1217",  "1307",  "1400",  "1494",  "1598",  "1684", "1773", "1863",  "1949",  "2040",  "2133" , "2225",  "2318",
"0072",  "0162",  "0259", "0339",  "0432",  "0515",  "0600",  "0676", "0768",  "0861", "0952",  "1031" ,"1128",  "1219",  "1308",  "1401",  "1495",  "1599",  "1685", "1775", "1864",  "1953",  "2041",  "2134" , "2226",  "2319",
"0074",  "0163",  "0260", "0340",  "0433",  "0516",  "0601",  "0677", "0769",  "0862", "0953",  "1034" ,"1129",  "1220",  "1311",  "1402",  "1498",  "1600",  "1686", "1776", "1865",  "1954",  "2042",  "2135" , "2228",  "2321",
"0076",  "0165",  "0261", "0341",  "0434",  "0517",  "0602",  "0678", "0773",  "0864", "0954",  "1035" ,"1130",  "1221",  "1313",  "1404",  "1499",  "1601",  "1687", "1777", "1866",  "1956",  "2044",  "2136" , "2231",  "2322",
"0079",  "0166",  "0262", "0342",  "0435",  "0518",  "0603",  "0679", "0777",  "0865", "0955",  "1036" ,"1131",  "1222",  "1314",  "1405",  "1500",  "1603",  "1688", "1778", "1867",  "1958",  "2045",  "2137" , "2232",  "2323",
"0080",  "0167",  "0263", "0344",  "0436",  "0520",  "0605",  "0680", "0778",  "0868", "0956",  "1037" ,"1132",  "1223",  "1317",  "1406",  "1502",  "1604",  "1689", "1779", "1868",  "1959",  "2046",  "2138" , "2233",  "2324",
"0081",  "0168",  "0268", "0348",  "0437",  "0521",  "0606",  "0681", "0779",  "0869", "0958",  "1039" ,"1134",  "1225",  "1321",  "1407",  "1503",  "1605",  "1691", "1780", "1869",  "1960",  "2047",  "2140" , "2234",  "2325",
"0082",  "0172",  "0269", "0349",  "0438",  "0522",  "0607",  "0682", "0780",  "0870", "0959",  "1040" ,"1135",  "1226",  "1322",  "1408",  "1504",  "1607",  "1694", "1781", "1870",  "1961",  "2048",  "2141" , "2236",  "2326",
"0083",  "0174",  "0270", "0350",  "0439",  "0523",  "0608",  "0683", "0781",  "0871", "0960",  "1041" ,"1136",  "1228",  "1323",  "1409",  "1505",  "1608",  "1695", "1782", "1871",  "1962",  "2051",  "2143" , "2237",
"0084",  "0175",  "0271", "0351",  "0440",  "0525",  "0609",  "0684", "0782",  "0872", "0961",  "1043" ,"1138",  "1229",  "1327",  "1410",  "1506",  "1610",  "1699", "1783", "1872",  "1964",  "2052",  "2144" , "2238",
"0085",  "0176",  "0272", "0354",  "0441",  "0526",  "0611",  "0687", "0784",  "0874", "0962",  "1044" ,"1139",  "1230",  "1328",  "1411",  "1508",  "1611",  "1701", "1784", "1873",  "1965",  "2054",  "2146" , "2241",
"0086",  "0178",  "0274", "0356",  "0443",  "0527",  "0612",  "0688", "0785",  "0876", "0963",  "1047" ,"1140",  "1232",  "1329",  "1413",  "1509",  "1612",  "1703", "1785", "1874",  "1966",  "2055",  "2147" , "2242",
"0087",  "0179",  "0276", "0357",  "0444",  "0530",  "0613",  "0689", "0786",  "0878", "0964",  "1048" ,"1141",  "1233",  "1330",  "1414",  "1510",  "1613",  "1705", "1786", "1875",  "1967",  "2056",  "2148" , "2245",
"0088",  "0181",  "0277", "0359",  "0447",  "0531",  "0614",  "0691", "0787",  "0879", "0965",  "1054" ,"1142",  "1236",  "1332",  "1415",  "1511",  "1614",  "1708", "1787", "1877",  "1968",  "2057",  "2150" , "2248",
"0090",  "0185",  "0278", "0360",  "0448",  "0533",  "0615",  "0692", "0788",  "0880", "0966",  "1056" ,"1143",  "1237",  "1333",  "1416",  "1512",  "1615",  "1710", "1790", "1878",  "1969",  "2058",  "2151" , "2249",
"0091",  "0186",  "0279", "0361",  "0449",  "0534",  "0616",  "0694", "0789",  "0881", "0968",  "1058" ,"1145",  "1239",  "1335",  "1417",  "1515",  "1616",  "1711", "1791", "1880",  "1971",  "2059",  "2152" , "2251"
]
for video_id in video_ids:
    process_video(video_id)


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Already processed
0001 Alread

0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Already processed
0094 Alread

0187:   0%|                                                                                                                          | 0/35 [00:00<?, ?it/s]

0187 Already processed
0187 Already processed
0187 Already processed
0187 Already processed
0187 Already processed
0187 Already processed
0187 Already processed
0187 Already processed
0187 Already processed
0187 Already processed
0187 Already processed


...

✅ All frames of video 1880 processed → ../project/output/


✅ All frames of video 1971 processed → ../project/output/


✅ All frames of video 2059 processed → ../project/output/


✅ All frames of video 2152 processed → ../project/output/


✅ All frames of video 2251 processed → ../project/output/
